# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install citipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys2 import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df =pd.read_csv('../WeatherPy/citiesdata.csv')
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,62.01,73,92,1.99,AU,2021-04-30 01:38:26
1,iracoubo,5.4802,-53.2011,76.28,87,100,7.11,GF,2021-04-30 01:38:26
2,puerto ayora,-0.7393,-90.3518,74.68,84,85,7.25,EC,2021-04-30 01:38:26
3,tottori,35.5000,134.2333,64.40,59,75,5.75,JP,2021-04-30 01:38:27
4,east london,-33.0153,27.9116,59.00,94,20,9.22,ZA,2021-04-30 01:38:27


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
# HINT: be sure to handle NaN values
weather_df = weather_df.dropna()
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[['Lat','Lng']].astype(float)
# Convert Humidity to float and store
weight = weather_df['Humidity'].astype(float)

In [5]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1.5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#A max temperature lower than 80 degrees but higher than 50.
condition_a = (weather_df['Max Temp'] < 77) & (weather_df['Max Temp'] > 30)
#Wind speed less than 12 mph.
condition_b = (weather_df['Wind Speed'] < 30)
#Zero cloudiness.
condition_c = (weather_df['Cloudiness'] > 50)
#Drop any rows that don't contain all three conditions. 
weather_condition = condition_a & condition_b & condition_c
#You want to be sure the weather is ideal.
fitting_weather = weather_df[weather_condition]
fitting_weather = fitting_weather.dropna(how='any',axis=0)
fitting_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,62.01,73,92,1.99,AU,2021-04-30 01:38:26
1,iracoubo,5.4802,-53.2011,76.28,87,100,7.11,GF,2021-04-30 01:38:26
2,puerto ayora,-0.7393,-90.3518,74.68,84,85,7.25,EC,2021-04-30 01:38:26
3,tottori,35.5000,134.2333,64.40,59,75,5.75,JP,2021-04-30 01:38:27
6,guasdualito,7.2424,-70.7324,75.20,100,100,1.14,VE,2021-04-30 01:38:28
...,...,...,...,...,...,...,...,...,...
550,colares,38.7992,-9.4469,57.00,81,100,3.00,PT,2021-04-30 01:42:35
554,vilcun,-38.6500,-72.2333,51.80,100,90,2.30,CL,2021-04-30 01:42:36
556,havre-saint-pierre,50.2334,-63.5986,41.00,87,90,9.22,CA,2021-04-30 01:42:37
558,entre rios,-11.9419,-38.0844,69.10,97,56,3.67,BR,2021-04-30 01:42:38


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
fitting_weather['Hotel Name'] = ""
fitting_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel name,Hotel Name
0,busselton,-33.6500,115.3333,62.01,73,92,1.99,AU,2021-04-30 01:38:26,,
1,iracoubo,5.4802,-53.2011,76.28,87,100,7.11,GF,2021-04-30 01:38:26,,
2,puerto ayora,-0.7393,-90.3518,74.68,84,85,7.25,EC,2021-04-30 01:38:26,,
3,tottori,35.5000,134.2333,64.40,59,75,5.75,JP,2021-04-30 01:38:27,,
6,guasdualito,7.2424,-70.7324,75.20,100,100,1.14,VE,2021-04-30 01:38:28,,
...,...,...,...,...,...,...,...,...,...,...,...
550,colares,38.7992,-9.4469,57.00,81,100,3.00,PT,2021-04-30 01:42:35,,
554,vilcun,-38.6500,-72.2333,51.80,100,90,2.30,CL,2021-04-30 01:42:36,,
556,havre-saint-pierre,50.2334,-63.5986,41.00,87,90,9.22,CA,2021-04-30 01:42:37,,
558,entre rios,-11.9419,-38.0844,69.10,97,56,3.67,BR,2021-04-30 01:42:38,,


In [29]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Store into variable named hotel_df.
hotel_df = fitting_weather[['City','Country','Lat','Lng','Hotel Name']]

#Set parameters to search for hotels with 5000 meters.

# create a params dict that will be updated with new city each iteration


# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():
        
        lat = row['Lat']
        lng = row['Lng']
        # update address key value
        params = {
            'radious': 5000,
            'location':f'{lat},{lng}',
            'rankby': 'distance',
            'keyword':'Hotel',
            'key':g_key
        }
        
        # make request
        response = requests.get(base_url, params=params)
        response = response.json()
        try:
            print(response["results"][0]["name"])
            hotel_df.loc[index, 'Hotel Name'] = response["results"][0]["name"]
        except:
            print('Missing field/result... skipping.')
        
    

Jacaranda Guest House
L'oiseau de Paradis
Hotel El Descanso del Guía
Tottori City Hotel
LLANO GRANDE HOTEL
Keleti Beach Resort
Hotel Iberia de Coquimbo
Arktika
Hotel Tshalukambo
Above Tide Motel
Firlane House
Aires del Beagle Apartaments
Casa da Madrinha Graciosa
Pousada Encanto Abrolhos
Post-Plaza Guesthouse
Xingfu Hostel
DoubleTree by Hilton Jamestown
Comfort Inn Glenmont - Albany South
CasaHost
The Lookout Guest House
Tulipoka Lodges
The Floating B&B
Apera Beach Front Bungalows
Hotel Captain Cook
Hotel Spresiano
Sberbank Rossii
Aabi's at Byron Boutique Accommodation
Grande Hotel (PR)
Café Hôtel de la Gare
Hotel Mead & Conference Center
Missing field/result... skipping.
Hostal Buenavista Patagonia
Okean
Tourist hotel
Hotel Ceyhan
Hotel Plaza Lebu
Sberbank Rossii
Missing field/result... skipping.
Puffin Nest Capsule Hostel
Kaya Bungalow Evleri
Hotel Belisario Jara
Bluff Lodge & Guest House
Casa Bara Toplita
HOTEL R. SOARES
Chonpines Hotelsa Zhuanghe Huanghai Street
Missing field/resul

In [32]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,busselton,AU,-33.6500,115.3333,Jacaranda Guest House
1,iracoubo,GF,5.4802,-53.2011,L'oiseau de Paradis
2,puerto ayora,EC,-0.7393,-90.3518,Hotel El Descanso del Guía
3,tottori,JP,35.5000,134.2333,Tottori City Hotel
6,guasdualito,VE,7.2424,-70.7324,LLANO GRANDE HOTEL


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content=[f"{name}" for name in hotel_info])
fig.add_layer(heat_layer)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))